In [1]:
import setup
setup.init()

In [3]:
resources = [
    {
        "name": "Documents",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [ ]:
# read_only_permissions = [
#     "document:read",
#     "movie_disovery:search",
#     "movie_disovery:detail",
# ]


In [7]:
roles= [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": {
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail",
        }
    },
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": read_only_permissions,
    }
]